# FER2013 Emotion Recognition - Advanced Implementation

This notebook implements a state-of-the-art emotion recognition system using the FER2013 dataset.

## Improvements:
- Comprehensive data exploration and visualization
- Advanced data augmentation techniques
- Multiple model architectures (EfficientNet)
- Better preprocessing and normalization
- Ensemble methods
- Real-time emotion detection interface

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras imports
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Dense, Dropout, GlobalAveragePooling2D, Input, BatchNormalization,
    Conv2D, MaxPooling2D, Flatten, Activation, Add, AveragePooling2D,
    LayerNormalization, SeparableConv2D  # Added SeparableConv2D for efficiency
)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, 
    TensorBoard, LearningRateScheduler
)
from tensorflow.keras.regularizers import l2
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")

# --- FIXED Configuration for FER2013 ---
IMG_WIDTH, IMG_HEIGHT = 48, 48
BATCH_SIZE = 64  # Increased for M3 Pro efficiency
COLOR_MODE = 'grayscale'  # FIXED: FER2013 is grayscale
NUM_CLASSES = 7
EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

# Dataset paths
BASE_DATASET_DIR = os.path.expanduser("~/Python/archive")
TRAIN_DIR = os.path.join(BASE_DATASET_DIR, "train")
TEST_DIR = os.path.join(BASE_DATASET_DIR, "test")

# --- Mac M3 Pro Optimization ---
# Disable mixed precision for stability on Apple Silicon
print("\nConfiguring for Apple Silicon M3 Pro...")

# Set memory growth for Apple Silicon
if tf.config.list_physical_devices('GPU'):
    print("Metal GPU detected")
else:
    print("Running on CPU")

# Verify dataset directories
if os.path.exists(TRAIN_DIR) and os.path.exists(TEST_DIR):
    print(f"\nDataset directories found:")
    print(f"  Training: {TRAIN_DIR}")
    print(f"  Testing: {TEST_DIR}")
else:
    raise FileNotFoundError("Dataset directories not found!")

In [ ]:
# Cell 2: Data Exploration and Visualization

print("=== Data Exploration ===")

# Function to count images per class
def count_images_per_class(directory):
    class_counts = {}
    for class_name in os.listdir(directory):
        class_path = os.path.join(directory, class_name)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))])
            class_counts[class_name] = count
    return class_counts

# Count images
train_counts = count_images_per_class(TRAIN_DIR)
test_counts = count_images_per_class(TEST_DIR)

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Training data distribution
classes = list(train_counts.keys())
train_values = [train_counts[c] for c in classes]
test_values = [test_counts[c] for c in classes]

x = np.arange(len(classes))
width = 0.35

ax1.bar(x - width/2, train_values, width, label='Train', alpha=0.8)
ax1.bar(x + width/2, test_values, width, label='Test', alpha=0.8)
ax1.set_xlabel('Emotion Class')
ax1.set_ylabel('Number of Images')
ax1.set_title('Distribution of Images Across Classes')
ax1.set_xticks(x)
ax1.set_xticklabels(classes, rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Pie chart for training data
ax2.pie(train_values, labels=classes, autopct='%1.1f%%', startangle=90)
ax2.set_title('Training Data Class Distribution')

plt.tight_layout()
plt.show()

# Print statistics
print("\nDataset Statistics:")
print(f"Total training images: {sum(train_values)}")
print(f"Total test images: {sum(test_values)}")
print("\nClass distribution:")
for cls in classes:
    print(f"  {cls}: Train={train_counts[cls]}, Test={test_counts[cls]}")

# Calculate class imbalance ratio
max_class = max(train_values)
min_class = min(train_values)
imbalance_ratio = max_class / min_class
print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}")

# Display sample images from each class
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.ravel()

for idx, emotion in enumerate(classes[:7]):
    emotion_path = os.path.join(TRAIN_DIR, emotion)
    sample_image = os.listdir(emotion_path)[0]
    img_path = os.path.join(emotion_path, sample_image)
    
    img = load_img(img_path, color_mode='grayscale')
    img_array = img_to_array(img)
    
    axes[idx].imshow(img_array.squeeze(), cmap='gray')
    axes[idx].set_title(f'{emotion.capitalize()}\n({train_counts[emotion]} images)')
    axes[idx].axis('off')

# Hide the last subplot if we have 7 emotions
if len(classes) == 7:
    axes[7].axis('off')

plt.suptitle('Sample Images from Each Emotion Class', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Cell 3: Advanced Data Preprocessing and Augmentation

print("=== Fixed Data Preprocessing for FER2013 ===")

# Proper preprocessing for FER2013
def preprocess_input_fer2013(x):
    """Preprocessing optimized for FER2013 grayscale images"""
    # Simple normalization works best for FER2013
    x = x / 255.0
    return x

# Optimized augmentation for facial expressions
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_fer2013,
    rotation_range=10,  # Reduced - faces don't rotate much
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,  # Reduced shear
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.15
)

# Only preprocessing for validation/test
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_fer2013
)

# Create data generators
print("\nCreating data generators...")

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode=COLOR_MODE,  # Now grayscale
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode=COLOR_MODE,  # Now grayscale
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    color_mode=COLOR_MODE,  # Now grayscale
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Calculate class weights
class_labels = train_generator.classes
class_weights_array = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(class_labels),
    y=class_labels
)
CLASS_WEIGHTS = dict(enumerate(class_weights_array))

print(f"\nData generators created:")
print(f"  Training samples: {train_generator.samples}")
print(f"  Validation samples: {validation_generator.samples}")
print(f"  Test samples: {test_generator.samples}")
print(f"\nClass weights: {CLASS_WEIGHTS}")

In [ ]:
# Cell 4: Custom CNN Architecture Optimized for FER2013

print("=== Building Custom CNN for FER2013 ===")

def create_fer2013_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 1), 
                       num_classes=NUM_CLASSES,
                       l2_reg=0.0001):
    """
    Custom CNN optimized for FER2013 48x48 grayscale images
    Efficient architecture for M3 Pro Mac
    """
    
    model = Sequential([
        # Block 1 - Initial feature extraction
        Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(l2_reg), 
               input_shape=input_shape),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(64, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 2 - Deeper features
        Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(128, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 3 - Complex patterns
        Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        Conv2D(256, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.25),
        
        # Block 4 - High-level features (using SeparableConv2D for efficiency)
        SeparableConv2D(512, (3, 3), padding='same', kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        GlobalAveragePooling2D(),  # More efficient than Flatten
        
        # Classification head
        Dense(256, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        
        Dense(128, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    return model

model = create_mini_xception()  # or create_fer2013_cnn() for standard CNN

# Compile with optimized settings for M3 Pro
optimizer = Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
)

model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("Model created successfully!")
print(f"\nModel Summary:")
model.summary()

# Save model architecture
tf.keras.utils.plot_model(
    model, 
    to_file='fer2013_model_architecture.png',
    show_shapes=True,
    show_layer_names=True,
    dpi=100
)

In [ ]:
# Cell 5: Optimized Training for M3 Pro

print("=== Optimized Training Setup ===")

# Cosine annealing with warm restarts
def cosine_annealing_with_warmup(epoch, lr):
    """Cosine annealing with initial warmup"""
    warmup_epochs = 5
    max_epochs = 50
    initial_lr = 0.001
    min_lr = 1e-6
    
    if epoch < warmup_epochs:
        return initial_lr * (epoch + 1) / warmup_epochs
    else:
        progress = (epoch - warmup_epochs) / (max_epochs - warmup_epochs)
        return min_lr + (initial_lr - min_lr) * 0.5 * (1 + np.cos(np.pi * progress))

# Optimized callbacks for FER2013
callbacks = [
    ModelCheckpoint(
        'best_fer2013_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    EarlyStopping(
        monitor='val_accuracy',
        patience=20,
        restore_best_weights=True,
        verbose=1
    ),
    
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,
        min_lr=1e-6,
        verbose=1
    ),
    
    LearningRateScheduler(cosine_annealing_with_warmup, verbose=0)
]

# Training configuration optimized for M3 Pro
EPOCHS = 50  # Reduced from 80 total

print(f"\nTraining configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Initial learning rate: {optimizer.learning_rate.numpy()}")

# Train the model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=CLASS_WEIGHTS,
    verbose=1,
    workers=4,  # Optimize for M3 Pro
    use_multiprocessing=False  # Better stability on Mac
)

print("\nTraining completed!")

In [ ]:
# Cell 6: Fine-tuning Phase

print("\n=== Phase 2: Fine-tuning ===")

# Unfreeze the base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = len(base_model.layers) - 20

# Freeze all the layers before fine_tune_at
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

print(f"Unfreezing top {len(base_model.layers) - fine_tune_at} layers of base model")
print(f"Total trainable layers: {len([l for l in model.layers if l.trainable])}")

# Recompile with lower learning rate
fine_tune_lr = 1e-5
model.compile(
    optimizer=Adam(learning_rate=fine_tune_lr),
    loss='categorical_crossentropy',
    metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Continue training
history_fine = model.fit(
    train_generator,
    epochs=INITIAL_EPOCHS + FINE_TUNE_EPOCHS,
    initial_epoch=len(initial_history['loss']),
    validation_data=validation_generator,
    callbacks=callbacks,
    class_weight=CLASS_WEIGHTS,
    verbose=1
)

# Combine histories
history = {}
for key in initial_history.keys():
    history[key] = initial_history[key] + history_fine.history[key]

print("\nFine-tuning completed!")

In [ ]:
# Cell 7: Comprehensive Model Evaluation

print("=== Model Evaluation ===")

# Load best model
best_model = tf.keras.models.load_model('best_emotion_model.keras')

# Evaluate on test set
test_loss, test_acc, test_precision, test_recall = best_model.evaluate(
    test_generator,
    verbose=1
)

print(f"\nTest Results:")
print(f"  Loss: {test_loss:.4f}")
print(f"  Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"  Precision: {test_precision:.4f}")
print(f"  Recall: {test_recall:.4f}")
print(f"  F1-Score: {2 * (test_precision * test_recall) / (test_precision + test_recall):.4f}")

# Generate predictions
y_pred = best_model.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Ensure alignment
y_true = y_true[:len(y_pred_classes)]

# Classification report
print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred_classes, 
                          target_names=EMOTION_LABELS,
                          digits=3))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=EMOTION_LABELS,
            yticklabels=EMOTION_LABELS,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Emotion Recognition Model', fontsize=16)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.show()

# Calculate per-class accuracy
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
plt.figure(figsize=(10, 6))
bars = plt.bar(EMOTION_LABELS, per_class_accuracy)
plt.title('Per-Class Accuracy', fontsize=16)
plt.xlabel('Emotion', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.ylim(0, 1)

# Add value labels on bars
for bar, acc in zip(bars, per_class_accuracy):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{acc:.2f}', ha='center', va='bottom')

plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

In [ ]:
# Cell 8: Training History Visualization

print("=== Training History Visualization ===")

# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history['accuracy'], label='Training Accuracy')
axes[0, 0].plot(history['val_accuracy'], label='Validation Accuracy')
axes[0, 0].axvline(x=INITIAL_EPOCHS, color='red', linestyle='--', label='Start Fine-tuning')
axes[0, 0].set_title('Model Accuracy', fontsize=14)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Loss
axes[0, 1].plot(history['loss'], label='Training Loss')
axes[0, 1].plot(history['val_loss'], label='Validation Loss')
axes[0, 1].axvline(x=INITIAL_EPOCHS, color='red', linestyle='--', label='Start Fine-tuning')
axes[0, 1].set_title('Model Loss', fontsize=14)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Learning Rate
if 'lr' in history:
    axes[1, 0].plot(history['lr'], label='Learning Rate')
    axes[1, 0].set_title('Learning Rate Schedule', fontsize=14)
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Learning Rate')
    axes[1, 0].set_yscale('log')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# Precision and Recall
if 'precision' in history:
    axes[1, 1].plot(history['precision'], label='Training Precision')
    axes[1, 1].plot(history['val_precision'], label='Validation Precision')
    axes[1, 1].plot(history['recall'], label='Training Recall', linestyle='--')
    axes[1, 1].plot(history['val_recall'], label='Validation Recall', linestyle='--')
    axes[1, 1].set_title('Precision and Recall', fontsize=14)
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Score')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Training History', fontsize=16)
plt.tight_layout()
plt.show()

# Print final metrics
print("\nFinal Training Metrics:")
print(f"  Best Validation Accuracy: {max(history['val_accuracy']):.4f}")
print(f"  Best Validation Loss: {min(history['val_loss']):.4f}")
print(f"  Final Training Accuracy: {history['accuracy'][-1]:.4f}")
print(f"  Final Validation Accuracy: {history['val_accuracy'][-1]:.4f}")

In [ ]:
# Cell 9: Fixed Real-time Emotion Detection

print("=== Emotion Detection Interface ===")

def preprocess_image(image_path, target_size=(48, 48)):
    """
    Preprocess a single image for prediction
    """
    # Load as grayscale
    img = load_img(image_path, target_size=target_size, color_mode='grayscale')
    img_array = img_to_array(img)
    
    # Normalize
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def predict_emotion(model, image_path, show_probabilities=True):
    """
    Predict emotion from an image file
    """
    # Preprocess image
    processed_image = preprocess_image(image_path)
    
    # Make prediction
    predictions = model.predict(processed_image, verbose=0)
    predicted_class = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class]
    
    # Get emotion label
    predicted_emotion = EMOTION_LABELS[predicted_class]
    
    # Display results
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # Show image
    img = load_img(image_path, color_mode='grayscale')
    ax1.imshow(img, cmap='gray')
    ax1.set_title(f'Predicted: {predicted_emotion} ({confidence*100:.1f}%)', fontsize=14)
    ax1.axis('off')
    
    # Show probability distribution
    if show_probabilities:
        bars = ax2.bar(EMOTION_LABELS, predictions[0])
        ax2.set_title('Emotion Probabilities', fontsize=14)
        ax2.set_xlabel('Emotion')
        ax2.set_ylabel('Probability')
        ax2.set_ylim(0, 1)
        plt.xticks(rotation=45)
        
        # Highlight predicted emotion
        bars[predicted_class].set_color('red')
        
        # Add value labels
        for i, (emotion, prob) in enumerate(zip(EMOTION_LABELS, predictions[0])):
            ax2.text(i, prob + 0.01, f'{prob:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()
    
    return predicted_emotion, confidence, predictions[0]

# Save the final model
best_model = tf.keras.models.load_model('best_fer2013_model.keras')
best_model.save('fer2013_emotion_detector_final.keras')
print("\nFinal model saved!")

In [ ]:
# Cell 10: Model Export and Deployment Preparation

print("=== Model Export for Deployment ===")

# Save model in multiple formats
# 1. SavedModel format (for TensorFlow Serving)
best_model.save('emotion_detector_savedmodel', save_format='tf')
print("Model saved in SavedModel format for TensorFlow Serving")

# 2. TFLite conversion for mobile deployment
converter = tf.lite.TFLiteConverter.from_keras_model(best_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

with open('emotion_detector.tflite', 'wb') as f:
    f.write(tflite_model)
print("Model converted to TFLite format for mobile deployment")

# 3. Save model configuration and weights separately
model_json = best_model.to_json()
with open('emotion_detector_architecture.json', 'w') as json_file:
    json_file.write(model_json)
best_model.save_weights('emotion_detector_weights.h5')
print("Model architecture and weights saved separately")

# Create a simple inference script
inference_script = '''import tensorflow as tf
import numpy as np
from PIL import Image

# Load model
model = tf.keras.models.load_model('fer2013_emotion_detector_final.keras')

# Emotion labels
EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

def predict_emotion(image_path):
    # Load and preprocess image
    img = Image.open(image_path).convert('RGB')
    img = img.resize((48, 48))
    img_array = np.array(img)
    img_array = (img_array - 127.5) / 127.5
    img_array = np.expand_dims(img_array, axis=0)
    
    # Predict
    predictions = model.predict(img_array)
    emotion_idx = np.argmax(predictions[0])
    emotion = EMOTION_LABELS[emotion_idx]
    confidence = predictions[0][emotion_idx]
    
    return emotion, confidence

# Example usage
if __name__ == "__main__":
    import sys
    if len(sys.argv) > 1:
        emotion, confidence = predict_emotion(sys.argv[1])
        print(f"Emotion: {emotion} (Confidence: {confidence:.2%})")
    else:
        print("Usage: python predict_emotion.py <image_path>")
'''

with open('predict_emotion.py', 'w') as f:
    f.write(inference_script)
print("\nInference script created: predict_emotion.py")

# Model summary for deployment
print("\n=== Deployment Summary ===")
print(f"Model input shape: {best_model.input_shape}")
print(f"Model output shape: {best_model.output_shape}")
print(f"Total parameters: {best_model.count_params():,}")
print(f"Model size (Keras): {os.path.getsize('fer2013_emotion_detector_final.keras') / 1024 / 1024:.2f} MB")
print(f"Model size (TFLite): {os.path.getsize('emotion_detector.tflite') / 1024 / 1024:.2f} MB")

print("\n✅ Model is ready for deployment!")
print("\nNext steps:")
print("1. Use 'fer2013_emotion_detector_final.keras' for Python applications")
print("2. Use 'emotion_detector.tflite' for mobile applications")
print("3. Use 'emotion_detector_savedmodel' for TensorFlow Serving")
print("4. Run 'python predict_emotion.py <image_path>' for quick predictions")